<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DBD_Engine3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
kB = 1.38e-23 # Boltzmann constant
e = 1.6e-19 # Electron charge
m = 9.11e-31 # Electron mass
n0 = 2.5e19 # Number density of air molecules
T0 = 300 # Temperature of air
vth = np.sqrt(2*kB*T0/m) # Thermal velocity

# Electric field
E = 1e3 # Electric field strength

# Rate coefficients (in m^3/s)
alpha_ion = 1.5e-20 # Ionization
alpha_att = 2e-20 # Attachment
alpha_el = 7.6e-20 # Elastic scattering
alpha_tot = alpha_ion + alpha_att + alpha_el # Total collision rate

# Spatial grid
L = 1e-3 # Length of simulation domain
dx = 1e-6 # Grid spacing
x = np.arange(0, L+dx, dx)
N = len(x)

# Time stepping
dt = 1e-11 # Time step
t = 0 # Current time
tmax = 1e-8 # Maximum simulation time

# Initial conditions
n = n0*np.ones(N) # Number density of air molecules
Te = T0*np.ones(N) # Electron temperature
J = np.zeros(N) # Electric current
Ee = np.zeros(N) # Electron energy

# Main loop
while t < tmax:
    # Calculate electric field
    E = -np.gradient(Ee, dx)
    
    # Calculate electron velocity
    ve = np.sqrt(2*e*Te/m)
    
    # Calculate electron mean free path
    lambda_e = ve/alpha_tot
    
    # Calculate electron collision frequency
    nu_e = alpha_tot*ve
    
    # Calculate electron collision time
    tau_e = 1/nu_e
    
    # Calculate momentum balance
    n[1:-1] += dt*(-np.gradient(J, dx)/e)
    
    # Calculate species balance
    n[1:-1] += dt*(-alpha_ion*n[1:-1]*n[1:-1] + alpha_att*Te[1:-1]*n[1:-1])
    
    # Calculate electron energy balance
    Te[1:-1] += dt*(-(Te[1:-1]-T0)/tau_e + e*E[1:-1]/(kB*Te[1:-1])*J[1:-1]/n[1:-1])
    
    # Calculate electric current
    J[1:-1] = e*n[1:-1]*ve[1:-1]*E[1:-1]
    
    # Calculate electron energy
    Ee[1:-1] = 0.5*m*ve[1:-1]*ve[1:-1]/e + kB*Te[1:-1]
    
    # Apply boundary conditions
    n[0] = n0
    n[-1] = n[-2]
    Te[0] = T0
    Te[-1] = Te[-2]
    J[0] = 0
    J[-1] = J[-2]
    Ee[0] = kB*T0/e
    Ee[-1] = Ee[-2]
    
    # Increment time
    t += dt

# Plot results
plt.figure()
plt.plot(x, n/n0, label='Number density')
plt.xlabel('Distance (m)')
plt.ylabel('Number density (normalized)')
plt.legend()

plt
